# 0. 导入必要的包

In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("./tools/")
import ucTools
%load_ext autoreload

# 1. 定义必要的函数

In [2]:
def calulate_distinct_EC(eclist):
    counter =0
    res = []
    for item in eclist:
        itemArray = item.split(',')
        res+=itemArray
    res = np.array(res, dtype=object).flatten()
    res = np.unique(res)
    return res

def get_ec_list(ec_table):
    ec_list = calulate_distinct_EC(ec_table.ec_number)
    ec_list = pd.DataFrame({'ec_number':ec_list})

    ec_list['c1'] = ec_list.ec_number.map(lambda x: x.split('.')[0].strip())
    ec_list['c2'] = ec_list.ec_number.map(lambda x: x.split('.')[1].strip())
    ec_list['c3'] = ec_list.ec_number.map(lambda x: x.split('.')[2].strip())
    ec_list['c4'] = ec_list.ec_number.map(lambda x: x.split('.')[3].strip())

    return ec_list

def split_multi_function_emzyme(multi_df):
    '''
        将多功能酶拆分成多行并返回
    '''
    splitecs = []
    for index, row in multi_df.iterrows():
        ecs= row.ec_number.split(',')
        for ec in ecs:
            row.ec_number = ec
            splitecs +=[list(row.values)]
    return pd.DataFrame(splitecs,columns=multi_df.columns.values.tolist())

# 2. Sprot
## 2.1 加载数据

In [3]:
sprot = pd.read_csv('./data/sprot_with_ec.tsv', sep='\t',names=['id', 'ec_number', 'date_integraged','date_sequence_update','date_annotation_update','seq']) #读入文件
sprot.date_integraged = pd.to_datetime(sprot['date_integraged'])
sprot.date_sequence_update = pd.to_datetime(sprot['date_sequence_update'])
sprot.date_annotation_update = pd.to_datetime(sprot['date_annotation_update'])
sprot.head(3)

,id,ec_number,date_integraged,date_sequence_update,date_annotation_update,seq
0,Q6GZW6,3.6.4.-,2011-06-28,2004-07-19,2020-08-12,MDTSPYDFLKLYPWLSRGEADKGTLLDAFPGETFEQSLASDVAMRR...
1,Q6GZV6,2.7.11.1,2011-06-28,2004-07-19,2021-02-10,MATNYCDEFERNPTRNPRTGRTIKRGGPVFRALERECSDGAARVFP...
2,Q197B6,2.7.11.-,2009-06-16,2006-07-11,2020-08-12,MPLSVFAEEFAEKSVKRYIGQGLWLPCNLSDYYYYQEFHDEGGYGS...


## 2.2 统计独立的EC号， 并拆分等级

In [4]:
ec_list = get_ec_list(sprot)
ec_list.head(3)

,ec_number,c1,c2,c3,c4
0,1.-.-.-,1,-,-,-
1,1.1.1.,1,1,1,
2,1.1.1.-,1,1,1,-


## 2.3 统计每一级的EC号

In [5]:
L1 = ec_list['c1'].value_counts().sort_index()
L2 = ec_list['c2'].value_counts().sort_index()
L3 = ec_list['c3'].value_counts().sort_index()
L4 = ec_list['c4'].value_counts().sort_index()
print('Level 1 EC: {0} 个'.format(len(L1)))
print('Level 2 EC: {0} 个'.format(len(L2)))
print('Level 3 EC: {0} 个'.format(len(L3)))
print('Level 4 EC: {0} 个'.format(len(L4)))

Level 1 EC: 7 个
Level 2 EC: 25 个
Level 3 EC: 32 个
Level 4 EC: 417 个


In [6]:
sprot_ec_konwn = ec_list[(ec_list.c1 !='-') & (ec_list.c2 !='-') & (ec_list.c3!='-') & (ec_list.c4!='-')]
print('Sprot 共有：「{0}」个独立的EC NUMBER.'.format(ec_list.shape[0]))
print('Sprot 共有：「{0}」个不含未知项的EC NUMBER.'.format(sprot_ec_konwn.shape[0]))

Sprot 共有：「6312」个独立的EC NUMBER.
Sprot 共有：「6005」个不含未知项的EC NUMBER.


## 2.4 将不含未知项EC号的数据保存

In [7]:
sprot_full_ec = sprot[~sprot['ec_number'].str.contains('-')]
sprot_full_ec.to_csv('./data/sprot_full_ec.tsv',index=False, sep='\t',header=0)

# 3. Trembl

In [11]:
trembl = pd.read_csv('./data/trembl_with_ec.tsv', sep='\t',names=['id', 'ec_number', 'date_integraged','date_sequence_update','date_annotation_update','seq']) #读入文件
trembl.date_integraged = pd.to_datetime(trembl['date_integraged'])
trembl.date_sequence_update = pd.to_datetime(trembl['date_sequence_update'])
trembl.date_annotation_update = pd.to_datetime(trembl['date_annotation_update'])
trembl.head(3)

,id,ec_number,date_integraged,date_sequence_update,date_annotation_update,seq
0,O08452,3.2.1.1,01-JUL-1997,01-JUL-1997,07-OCT-2020,MNIKKLTPLLTLLLFFIVLASPVSAAKYLELEEGGVIMQAFYWDVP...
1,H9BV67,2.8.4.1,16-MAY-2012,16-MAY-2012,07-OCT-2020,AMQIGMSFIGAYKMCAGEAAVADLAFAAKHAGVIQMADILPARRAR...
2,L0AQC3,2.8.4.1,06-MAR-2013,06-MAR-2013,07-OCT-2020,AAVADLAFAAKHAGVIQMGDILPARRARGPNEPGGIKFGHFADMIQ...


## 3.2 统计独立的EC号， 并拆分等级

In [16]:
ec_list = get_ec_list(trembl)
ec_list.head(3)

,ec_number,c1,c2,c3,c4
0,1.-.-.-,1,-,-,-
1,1.1.-.-,1,1,-,-
2,1.1.1.,1,1,1,


## 3.3 统计每一级的EC号

In [17]:
L1 = ec_list['c1'].value_counts().sort_index()
L2 = ec_list['c2'].value_counts().sort_index()
L3 = ec_list['c3'].value_counts().sort_index()
L4 = ec_list['c4'].value_counts().sort_index()
print('Level 1 EC: {0} 个'.format(len(L1)))
print('Level 2 EC: {0} 个'.format(len(L2)))
print('Level 3 EC: {0} 个'.format(len(L3)))
print('Level 4 EC: {0} 个'.format(len(L4)))

Level 1 EC: 7 个
Level 2 EC: 25 个
Level 3 EC: 32 个
Level 4 EC: 407 个


In [18]:
trembl_ec_konwn = ec_list[(ec_list.c1 !='-') & (ec_list.c2 !='-') & (ec_list.c3!='-') & (ec_list.c4!='-')]
print('Sprot 共有：「{0}」个独立的EC NUMBER.'.format(ec_list.shape[0]))
print('Sprot 共有：「{0}」个不含未知项的EC NUMBER.'.format(trembl_ec_konwn.shape[0]))

Sprot 共有：「8285」个独立的EC NUMBER.
Sprot 共有：「7828」个不含未知项的EC NUMBER.


## 3.4 将不含未知项EC号的数据保存

In [19]:
trembl_full_ec = trembl[~trembl['ec_number'].str.contains('-')]
trembl_full_ec.to_csv('./data/trembl_full_ec.tsv',index=False, sep='\t', )

# 4.不同数据中EC号比对

In [219]:
in_trembl_notin_sprot = trembl_ec_konwn[~trembl_ec_konwn.ec_number.isin(sprot_ec_konwn.ec_number)]
in_sprot_notin_trembl = sprot_ec_konwn[~sprot_ec_konwn.ec_number.isin(trembl_ec_konwn.ec_number)]
print('Trembl中出现的非空EC号个数为：{0}'.format(len(trembl_ec_konwn)))
print('Sprot中出现的非空EC号个数为：{0}'.format(len(sprot_ec_konwn)))

Trembl中出现的非空EC号个数为：7828
Sprot中出现的非空EC号个数为：6005


In [220]:
sprot[sprot_ec_konwn.ec_number.isin(trembl_ec_konwn.ec_number)]

,id,ec_number,date_integraged,date_sequence_update,date_annotation_update,seq
0,Q6GZW6,3.6.4.-,2011-06-28,2004-07-19,2020-08-12,MDTSPYDFLKLYPWLSRGEADKGTLLDAFPGETFEQSLASDVAMRR...
1,Q6GZV6,2.7.11.1,2011-06-28,2004-07-19,2021-02-10,MATNYCDEFERNPTRNPRTGRTIKRGGPVFRALERECSDGAARVFP...
2,Q197B6,2.7.11.-,2009-06-16,2006-07-11,2020-08-12,MPLSVFAEEFAEKSVKRYIGQGLWLPCNLSDYYYYQEFHDEGGYGS...
3,Q6GZS1,3.6.4.-,2011-06-28,2004-07-19,2020-08-12,MAKLLRLNAIDGDMPGAGEADLFTLAPGGKAYVPFAWGSRVLGCKP...
4,Q196Y0,3.6.1.-,2009-06-16,2006-07-11,2020-08-12,MDNETSTPDIFQWCVSPFSKITLKRSMEQRDIVEFRIDATILRQIF...
...,...,...,...,...,...,...
270231,Q4R4A2,"3.4.19.12, 3.4.19.12",2006-06-27,2005-07-19,2020-12-02,MLSCDICGETVTSEPDMKAHLIVHMENEIVCPFCKLSGVSYDEMCF...
270232,Q3T9Z9,3.4.19.12,2006-06-27,2006-06-27,2021-04-07,MLSCNICGETVNSEPDMKAHLIVHMENEIICPFCKLSGINYNEICF...
270233,Q5U2S3,3.4.19.12,2006-06-27,2004-12-07,2020-12-02,MLSCDICGETVTSEPDRKAHLIVHMENEIICPFCKLSGINYNEMCF...
270234,Q621J7,2.7.11.1,2005-04-12,2004-11-23,2021-02-10,MSGGKSGTKLSSFQNLQQIGQGGFGVVYSAQRENGEKVAIKKIGNA...


In [33]:
# trembl_full_ec = pd.read_csv('./data/trembl_full_ec.tsv', sep='\t',names=['id', 'ec_number', 'date_integraged','date_sequence_update','date_annotation_update','seq'])

trembl_full_ec['label_count'] = trembl_full_ec.ec_number.map(lambda x: x.count(',') + 1)
# query['seq'].map(lambda x : len(x))

<ipython-input-33-7154929d1774>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trembl_full_ec['label_count'] = trembl_full_ec.ec_number.map(lambda x: x.count(',') + 1)


In [35]:
trembl_full_ec[trembl_full_ec.label_count>1]

,id,ec_number,date_integraged,date_sequence_update,date_annotation_update,seq,label_count
45,A0A7G9YY52,"3.5.4.41, 3.5.4.31, 3.5.4.4, 3.5.4.28",2021-02-10,2021-02-10,2021-04-07,MINMSILIKNVLLEGKERNIYIEGNLIEAISKAGNGKEAEFVIDGK...,4
66,A0A7G9YWQ6,"4.2.1.136, 5.1.99.6",2021-02-10,2021-02-10,2021-04-07,MKEEKEEALSSAEMAAIDENCKFFGLSSAQLMENAGAGIANVIKRK...,2
237,A0A2D0TC99,"1.12.1.2, 1.12.99.6",2017-12-20,2017-12-20,2021-04-07,MVKIATTWLGGCSGCHISLLDLHEELLNLLENVELVHCPVLMDVKE...,2
260,A0A2D0TCA6,"1.12.99.6, 1.12.1.2",2017-12-20,2017-12-20,2021-04-07,MVKLSVEPVTRVEGHGKISVSFDDSGNLDKVRFHVVEVRGFEKFLE...,2
308,D1JA39,"5.4.2.2, 5.4.2.8",2010-01-19,2010-01-19,2020-10-07,MHTKKREISADIFRAYDIRGIYNDDLYPETMLKIGLGLGTYAKERS...,2
...,...,...,...,...,...,...,...
32826185,E0WDN6,"2.7.7.49, 3.1.26.13",2010-11-02,2010-11-02,2020-12-02,FFRENLAFQQREARKFPSEQTRTNSPTSRECGDGGRNTLXSEAGTE...,2
32826186,B9W4E1,"2.7.7.49, 3.1.26.13",2009-03-24,2009-03-24,2020-12-02,FFRENLAFQQREARKFSSEQTRTNSPTNGEPWGGGRDNLPCETGTE...,2
32826187,H6SH45,"2.7.7.49, 3.1.26.13",2012-04-18,2012-04-18,2020-12-02,FFRENLAFQQREARKFSSEQTRAISPTSRKLWDGGRDNPLPETGTE...,2
32826188,B9W4F0,"2.7.7.49, 3.1.26.13",2009-03-24,2009-03-24,2020-12-02,FFREGLAFQQREARQFSSEQTGTNSPTSRELRDRGRDNLLSEAGTE...,2


In [21]:
sprot_full_ec = pd.read_csv('./data/sprot_full_ec.tsv', sep='\t',names=['id', 'ec_number', 'date_integraged','date_sequence_update','date_annotation_update','seq'])

## 5. EC号分析

In [49]:
# 酶与非酶数据
sprot = pd.read_hdf('./data/emzyme_noemzyme_uncertain.h5',key='data')

In [50]:
sprot_single=sprot[~sprot.isMultiFunctional]
sprot_multi = split_multi_function_emzyme(sprot[sprot.isMultiFunctional])
sprot_all = pd.concat([sprot_single, sprot_multi], axis=0)

In [59]:
len(set(sprot_all[(sprot_all.ec_specific_level==4)].ec_number))

5894

In [195]:
sprot_all

,id,name,isemzyme,isMultiFunctional,functionCounts,ec_number,ec_specific_level,date_integraged,date_sequence_update,date_annotation_update,seq,seqlength
0,P02802,MT1_MOUSE,False,False,1,-,0,1986-07-21,1986-07-21,2021-04-07,MDPNCSCSTGGSCTCTSSCACKNCKCTSCKKSCCSCCPVGCSKCAQ...,61
1,P61852,SODC_DROSI,True,False,1,1.15.1.1,4,1986-07-21,2007-01-23,2020-12-02,MVVKAVCVINGDAKGTVFFEQESSGTPVKVSGEVCGLAKGLHGFHV...,153
2,P00925,ENO2_YEAST,True,False,1,4.2.1.11,4,1986-07-21,2007-01-23,2021-04-07,MAVSKVYARSVYDSRGNPTVEVELTTEKGVFRSIVPSGASTGVHEA...,437
3,P01228,FSHB_PIG,False,False,1,-,0,1986-07-21,1991-02-01,2020-12-02,MKSLQFCFLFCCWKAICCNSCELTNITITVEKEECNFCISINTTWC...,129
4,P01227,FSHB_SHEEP,False,False,1,-,0,1986-07-21,1989-10-01,2020-12-02,MKSVQFCFLFCCWRAICCRSCELTNITITVEKEECSFCISINTTWC...,129
...,...,...,...,...,...,...,...,...,...,...,...,...
587547,Q9SYJ8,SOK1_ARATH,False,False,1,-,0,2021-04-07,2000-05-01,2021-04-07,MESNGGGGEVRRVNLVYFLSRSGHVDHPHLLRVHHLSRNGVFLRDV...,372
587548,A0A1L8HX76,WDR18_XENLA,False,False,1,-,0,2021-04-07,2017-03-15,2021-04-07,MAAPLELVLSADAAGQVANCTVWEPVTGSVLLTYRGGNTSHRGLAV...,428
587549,A0A2K1JJ00,SOK1_PHYPA,False,False,1,-,0,2021-04-07,2018-03-28,2021-04-07,MAFEENFLSAQVLYQLSWNGKLEHPHIIDVEYASDQGGLRLRDVKE...,701
587550,Q06GJ0,HEXC_OSTFU,True,False,1,3.2.1.52,4,2021-04-07,2006-10-31,2021-04-07,MWSRRIPLFIFGVLVLILSVAAEDVVWRWSCDNGKCVKLKNDPRSS...,594


In [62]:
sprot_all.reset_index(drop=True, inplace=True)

In [120]:
full_length = pd.read_csv('./data/sprot_full.tsv', sep='\t', header=None)
full_length.columns =sprot_all.columns.values.tolist()

In [121]:
sprot_all = split_multi_function_emzyme(full_length)

In [127]:
sprot_all.to_excel('./data/sprot_splited_ec_full_length_data.xlsx', index=None)

In [ ]:
sprot_all

In [2]:
uctools =  ucTools.ucTools('172.16.25.20')
cnx_mimic = uctools.db_conn()

In [13]:
SPAN=[0, 5,10, 20, 30, 40, 50, 60, 100,150, 200,250, 300,350, 400, 450,500,550, 600, 700,800,900, 1000,1200, 1500, 3000, 40000]
sql = '''SELECT max(seqlength), min(seqlength) from tb_sprot;'''
MAXLEGTH = pd.read_sql_query(sql,cnx_mimic)['max'][0]

In [14]:
length_list =[]
for i in range(len(SPAN)-1):
    sql = ''' SELECT count(row_id) from tb_sprot where  seqlength BETWEEN {0} and {1}; '''.format(SPAN[i], SPAN[i+1])
    counts_value = pd.DataFrame(pd.read_sql_query(sql,cnx_mimic)).values[0][0]
    sql ='''SELECT count(*) from tb_sprot where  seqlength BETWEEN {0} and {1} and ec_number!='-';'''.format(SPAN[i], SPAN[i+1])
    has_ec = pd.DataFrame(pd.read_sql_query(sql,cnx_mimic)).values[0][0]
    sql ='''SELECT count(row_id) from tb_sprot WHERE seqlength BETWEEN {0} and {1} and ec_specific_level =4;'''.format(SPAN[i], SPAN[i+1])
    ec4 = pd.DataFrame(pd.read_sql_query(sql,cnx_mimic)).values[0][0]
    
    sql ='''SELECT count(row_id) from tb_sprot WHERE seqlength BETWEEN {0} and {1} and ec_specific_level =3;'''.format(SPAN[i], SPAN[i+1])
    ec3 = pd.DataFrame(pd.read_sql_query(sql,cnx_mimic)).values[0][0]
    
    sql ='''SELECT count(row_id) from tb_sprot WHERE seqlength BETWEEN {0} and {1} and ec_specific_level =2;'''.format(SPAN[i], SPAN[i+1])
    ec2 = pd.DataFrame(pd.read_sql_query(sql,cnx_mimic)).values[0][0]
    
    sql ='''SELECT count(row_id) from tb_sprot WHERE seqlength BETWEEN {0} and {1} and ec_specific_level =1;'''.format(SPAN[i], SPAN[i+1])
    ec1 = pd.DataFrame(pd.read_sql_query(sql,cnx_mimic)).values[0][0]
    
    sql ='''SELECT count(row_id) from tb_sprot WHERE seqlength BETWEEN {0} and {1} and ec_specific_level =0;'''.format(SPAN[i], SPAN[i+1])
    ec0 = pd.DataFrame(pd.read_sql_query(sql,cnx_mimic)).values[0][0]
    
    sql ='''SELECT count("isMultiFunctional") from tb_sprot WHERE seqlength BETWEEN {0} and {1} and "isMultiFunctional"=TRUE'''.format(SPAN[i], SPAN[i+1])
    multifun = pd.DataFrame(pd.read_sql_query(sql,cnx_mimic)).values[0][0]
    
    length_list +=[[str(SPAN[i])+'-'+str(SPAN[i+1]), int(counts_value), int(has_ec), ec4, ec3, ec2, ec1, ec0, multifun]]

In [15]:
aacc = pd.DataFrame(length_list, columns=['seqlength', 'seqcounts', 'have_ec_asigned', 'ec4', 'ec3', 'ec2', 'ec1', 'ec0', 'multifun'])
aacc['multi_percentage'] = aacc.multifun/aacc.seqcounts
aacc

,seqlength,seqcounts,have_ec_asigned,ec4,ec3,ec2,ec1,ec0,multifun,multi_percentage
0,0-5,70,5,5,0,0,0,65,0,0.000000
1,5-10,1157,91,71,16,3,1,1066,3,0.002593
2,10-20,2868,410,307,91,10,2,2458,12,0.004184
3,20-30,2300,337,259,72,4,2,1963,11,0.004783
4,30-40,4155,190,148,34,6,2,3965,13,0.003129
5,40-50,3656,150,122,24,3,1,3506,17,0.004650
6,50-60,5065,122,97,19,5,1,4943,7,0.001382
7,60-100,40713,4141,3188,843,100,10,36572,454,0.011151
8,100-150,62357,11094,8683,1507,793,111,51263,713,0.011434
9,150-200,62484,24576,19897,2987,1298,394,37908,2306,0.036905


In [17]:
data_unirep = pd.read_hdf('./data/emzyme_noemzyme_uncertain_with_unirep.h5',key='data')

In [28]:
data_unirep_sup = pd.read_hdf('./data/needUnirep_results.h5',key='data')

In [29]:
data_unirep_sub = data_unirep.iloc[:,12:1914]
data_unirep_sub.insert(0, 'seq', data_unirep.seq)
data_unirep_sub.insert(0, 'id', data_unirep.id)
data_unirep_sub

,id,seq,f1,f2,f3,f4,f5,f6,f7,f8,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
0,P02802,MDPNCSCSTGGSCTCTSSCACKNCKCTSCKKSCCSCCPVGCSKCAQ...,0.000455,-0.021951,0.027487,-0.000919,-0.228522,0.004351,-0.140352,-0.000167,...,-0.002212,0.119389,-0.019647,-0.016905,-0.532317,0.008659,-0.011296,0.001098,0.794472,0.210124
1,P02257,AASPQKRAASPRKSPKKSPRKSPKKKSPRKRKARSAAHPPVIDMIT...,-0.001693,0.002171,-0.007579,-0.027573,-0.161033,-0.030274,-0.644947,-0.000623,...,0.012868,0.003742,-0.048072,0.030513,-0.260276,-0.070536,0.002627,-0.002706,0.682809,0.455698
2,P01131,MRLAGWGLRWAIALLIAVGEAAVEDNCGRNEFQCQDGKCISYKWVC...,0.003249,-0.027980,0.021816,-0.031098,-0.452770,0.153213,-0.164409,-0.003949,...,0.002867,-0.181347,0.033731,0.023869,0.357923,0.098604,-0.009344,0.012374,-0.027197,0.109520
3,P00693,MGKNGSLCCFSLLLLLLLAGLASGHQVLFQGFNWESWKQSGGWYNM...,0.001718,0.191248,0.015037,-0.013877,-0.284253,0.006640,-0.425640,0.011292,...,0.074995,-0.098609,0.091042,0.331697,-0.060142,-0.067148,-0.039508,-0.044459,-0.043506,0.125468
4,P02255,MSDSAVATSASPVAAPPATVEKKVVQKKASGSAGTKAKKASATPSH...,-0.499300,-0.029771,0.034598,-0.001920,-0.011458,-0.006578,-0.713303,-0.000076,...,0.012111,-0.004334,-0.019177,0.002985,-0.078540,-0.003586,0.119747,-0.051190,0.989171,-0.141614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
453207,B1MCE2,MTENLQDMFESSYRGEAPEQLAARPPWSIGQPQPEILKLIEQGKVH...,0.002576,0.288816,0.040433,-0.860604,-0.261531,-0.047949,0.069571,-0.030992,...,-0.003655,0.000428,0.156489,0.578940,0.237726,0.643065,-0.007835,-0.027386,0.277779,0.073880
453208,K4GGT4,MVSEITHKSYPLHFVLFPFMAQGHMIPMVDIARLLAQRGVKITIVT...,0.002090,-0.034334,0.025631,-0.121367,-0.378588,-0.022098,-0.192928,0.002685,...,0.259137,-0.004872,0.101519,0.015377,-0.407052,0.009061,-0.010744,0.078966,0.038294,-0.191828
453209,K4GIP0,MVSEITHKSYPLHFVLFPFMAQGHMIPMVDIARLLAQRGVKITIVT...,0.002467,-0.039210,-0.119580,-0.229044,-0.452621,-0.028779,-0.165927,0.002078,...,0.208717,-0.007419,0.135355,0.003494,-0.331140,0.079519,-0.014041,0.082118,0.029292,-0.189873
453210,Q0S4D7,MELERTTIARMLFDRLGDDRLGVRTREQDWTWDEVVRESAARGAVA...,0.003614,0.007730,0.010237,-0.026348,-0.274717,0.551453,0.028152,-0.626685,...,0.006791,0.028422,0.392148,-0.153800,-0.048206,-0.042410,0.182518,0.004484,-0.015120,0.767742


In [64]:
unirep_mrege

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
0,P02802,0.000455,-0.021951,0.027487,-0.000919,-0.228522,0.004351,-0.140352,-0.000167,-0.001996,...,-0.002212,0.119389,-0.019647,-0.016905,-0.532317,0.008659,-0.011296,0.001098,0.794472,0.210124
1,P02257,-0.001693,0.002171,-0.007579,-0.027573,-0.161033,-0.030274,-0.644947,-0.000623,-0.011457,...,0.012868,0.003742,-0.048072,0.030513,-0.260276,-0.070536,0.002627,-0.002706,0.682809,0.455698
2,P01131,0.003249,-0.02798,0.021816,-0.031098,-0.45277,0.153213,-0.164409,-0.003949,-0.000905,...,0.002867,-0.181347,0.033731,0.023869,0.357923,0.098604,-0.009344,0.012374,-0.027197,0.10952
3,P00693,0.001718,0.191248,0.015037,-0.013877,-0.284253,0.00664,-0.42564,0.011292,-0.006632,...,0.074995,-0.098609,0.091042,0.331697,-0.060142,-0.067148,-0.039508,-0.044459,-0.043506,0.125468
4,P02255,-0.4993,-0.029771,0.034598,-0.00192,-0.011458,-0.006578,-0.713303,-0.000076,-0.002641,...,0.012111,-0.004334,-0.019177,0.002985,-0.07854,-0.003586,0.119747,-0.05119,0.989171,-0.141614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564633,X2ESR4,0.019841,-0.140942,-0.122081,-0.195677,-0.455057,-0.094894,-0.098551,-0.068419,-0.024038,...,0.254868,0.048895,0.035266,-0.048312,-0.627306,0.229116,-0.006243,0.082252,0.059653,-0.006167
564634,X2EVQ5,0.014427,-0.136654,-0.064704,-0.11865,-0.262763,0.01659,0.257547,-0.024436,-0.001509,...,0.05034,0.051047,-0.071278,-0.285197,-0.780717,0.278189,-0.003612,0.032204,0.039282,-0.01518
564635,X2JAU8,0.000277,-0.019201,0.025255,-0.002713,0.677392,0.005069,-0.737445,-0.000534,-0.001417,...,0.052981,0.161114,0.00942,0.013889,0.071083,-0.111909,-0.026444,0.055722,0.014272,0.069998
564636,X5CFH4,0.001009,0.065915,-0.068387,-0.036774,-0.630522,-0.076165,-0.477409,-0.018345,-0.05915,...,-0.008136,0.152099,0.036405,-0.043877,0.469758,0.355567,-0.015215,0.004849,-0.327303,-0.093813


In [31]:
unirep_mrege = pd.concat([data_unirep_sub, data_unirep_sup], axis=0)

In [34]:
unirep_mrege.reset_index(drop=True, inplace=True)

In [40]:
unirep_mrege.drop(columns=['seq'], inplace=True)

In [55]:
unirep_mrege.id = unirep_mrege.id.astype('str')

In [60]:
unit_thres = 10
print('开始精度采样，精度:{0}'.format(unit_thres))
unirep_mrege_low_specific = unirep_mrege.round(unit_thres)


开始精度采样，精度:10


In [70]:
unirep_mrege.to_feather('./data/sprot_unirep.feather')

In [71]:
aacdss = pd.read_feather('./data/sprot_unirep.feather')

In [65]:
h5.close()

In [6]:
sql = '''SELECT DISTINCT id, seq from tb_sprot;'''
sprot_data = pd.read_sql_query(sql,cnx_mimic)

In [ ]:
full_unirep = pd.concat

In [9]:
res = sprot_data.merge(data_unirep, how='left', on='id')

In [37]:
sub_unirep = res[res.f3.isnull()].iloc[:,0:2]

In [39]:
sub_unirep.reset_index(drop=True, inplace=True)

In [41]:
sub_unirep.to_csv('./data/needUnirep.tsv', sep='\t', index=None)

In [48]:
cc = [[]]
aa = np.array([[1,2,3,4]])
bb = np.array([[5,6,7,8]])

In [52]:
sub_unirepdata_unirep = pd.read_hdf('./data/needUnirep_results.h5',key='data')

In [53]:
sub_unirepdata_unirep

,id,seq,f1,f2,f3,f4,f5,f6,f7,f8,...,f1891,f1892,f1893,f1894,f1895,f1896,f1897,f1898,f1899,f1900
0,A0A023I4F1,MANPLISNHIGKHGKYTQAFLEQNGPGDARPTALDILKDNDRIDNM...,0.014085,0.101665,0.029611,-0.005112,-0.546254,-0.108098,-0.1883,-0.004759,...,0.314406,-0.107521,0.115466,-0.156177,-0.085792,0.017671,-0.00766,0.019784,0.04787,-0.168019
1,A0A023IWD9,MSDINATRLPVWIGYSPCVGDDCIALLTRGEGLC,0.001082,0.062657,0.032418,-0.015091,0.298175,0.050215,-0.080872,-0.010268,...,-0.015737,0.300234,0.032346,-0.037174,-0.099632,0.059644,0.038169,0.013604,0.060834,-0.013318
2,A0A023IWE0,MSDINATCLPAWLALCPCVGDDVNPTLTRGGT,0.00043,-0.038346,0.004205,-0.002733,-0.31856,0.009648,-0.545356,-0.010767,...,0.026578,0.277735,-0.077623,0.029638,-0.043806,-0.001656,-0.000065,-0.094784,0.126474,-0.044656
3,A0A023IWE1,MSDINGTRLPWLATCPCVGEDVNPTLSRGER,0.028526,-0.156609,0.00709,-0.001058,-0.113345,-0.009198,0.030857,-0.011372,...,0.046365,0.01031,-0.204787,0.069158,-0.042086,-0.006836,-0.008281,-0.022857,0.050033,-0.046287
4,A0A023IWE2,MSDINATRLPIWGIGCDPCVGDDVTAVLTRGEA,0.000638,-0.204822,0.132004,-0.002164,0.064247,0.016769,0.02738,-0.004623,...,-0.008681,0.066997,0.022463,-0.015224,-0.031084,0.044079,-0.003812,-0.179823,0.000034,-0.117388
5,A0A023IWE3,MSDINATRLPIWGIGCNPCVGDEVTALLTRGEA,0.001471,-0.169429,0.211058,-0.002332,0.08205,0.011161,0.143952,-0.005878,...,-0.006322,0.165025,0.024943,-0.012248,-0.042924,0.031335,-0.001963,-0.42983,0.00997,-0.060291
6,A0A023IWG1,MSDINATRLPVWIGYSPCVGDDAVALLNRGEG,0.001962,-0.090462,0.092444,-0.007539,0.698328,0.021192,0.175497,-0.036599,...,-0.059028,0.39248,0.022339,0.006877,-0.086979,-0.015568,-0.009134,0.083108,0.010185,-0.057846
7,A0A023IWG2,MSDINATRLPLILLAALGIPSDDADSTLTRGER,0.003097,-0.015594,0.021472,-0.003752,0.263266,-0.00857,0.455754,-0.02943,...,0.037626,-0.07445,-0.048108,0.062013,-0.071002,-0.334087,0.034729,-0.013522,0.046609,-0.106952
8,A0A023IWG3,MSDINATRLPIWGIGCDPCVGDEVTALLTRGEA,0.001217,-0.170734,0.165676,-0.002078,0.17696,0.013162,0.19071,-0.004867,...,-0.005877,0.102437,0.019365,-0.016683,-0.042423,0.032308,-0.000905,-0.355503,-0.000834,-0.083906
9,A0A023IWG4,MSDINATRLPIWGIGCNPSVGDEVTALLTSGEA,0.007777,-0.054018,0.230327,-0.003466,-0.01924,0.044634,-0.005027,-0.019493,...,-0.020354,0.242795,0.033322,0.009447,-0.063916,-0.012354,0.062669,0.171652,0.033847,-0.033774
